In [1]:
from ray import tune

In [2]:
from pathlib import Path
from mltrainer import ReportTypes


def train_rnn_with_attention_ray(config):
    import sys
    sys.path.append("/home/azureuser/machinelearning-melissa/networks")
    
    import torch
    from torch import optim
    from ray import tune
    from RNN import RecurrentNeuralNetworkWithAttention, ModelConfig
    from mltrainer import Trainer, TrainerSettings
    from mads_datasets import DatasetFactoryProvider, DatasetType
    from mltrainer.preprocessors import PaddedPreprocessor
    from mltrainer.metrics import Accuracy

    accuracy = Accuracy()
    loss_fn = torch.nn.CrossEntropyLoss()

    # Data
    factory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
    preprocessor = PaddedPreprocessor()
    streamers = factory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
    train = streamers["train"]
    valid = streamers["valid"]

    trainstreamer = train.stream()
    validstreamer = valid.stream()

    # Model
    model_config = ModelConfig(
        input_size=3,
        hidden_size=config["hidden_size"],
        num_layers=config["num_layers"],
        output_size=20,
        dropout=config["dropout"]
    )

    model = RecurrentNeuralNetworkWithAttention(model_config)

    settings = TrainerSettings(
        epochs=10, 
        metrics=[accuracy],
        train_steps=len(train),
        logdir=Path("ray_logs"),
        valid_steps=len(valid),
        reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD],
        earlystop_kwargs = {
            "save": False, # save every best model, and restore the best one
            "verbose": True,
            "patience": 5, # number of epochs with no improvement after which training will be stopped
        }
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    )

    trainer.loop()
    metrics, test_loss = trainer.evalbatches()
    print("!!!!!!!!!!!!!! DEBUG metrics dict:", metrics)
    tune.report(metrics=metrics)

In [3]:
search_space = {
    "hidden_size": tune.choice([64, 128, 256, 512]),
    "num_layers": tune.choice([1, 2, 3,4, 5]),
    "dropout": tune.uniform(0.0, 0.5),
    "lr": tune.loguniform(1e-4, 1e-2),
}

In [4]:
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

scheduler = ASHAScheduler(
    metric="Accuracy",  # of "accuracy" als je dat logt
    mode="max",  # of "max" als je accuracy logt
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    metric_columns=["loss", "accuracy", "training_iteration"]
)

result = tune.run(
    train_rnn_with_attention_ray,
    config=search_space,
    num_samples=10,  # aantal combinaties om te testen
    scheduler=scheduler,
    progress_reporter=reporter,
    resources_per_trial={"cpu": 1, "gpu": 0}  # of {"gpu": 1} als je GPU gebruikt
)


2025-06-04 20:04:35,311	INFO worker.py:1888 -- Started a local Ray instance.
2025-06-04 20:04:36,482	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2025-06-04 20:04:36,486	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2025-06-04 20:04:36 (running for 00:00:00.24)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0/4 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-06-04_20-04-33_195499_39160/artifacts/2025-06-04_20-04-36/train_rnn_with_attention_ray_2025-06-04_20-04-36/driver_artifacts
Number of trials: 10/10 (10 PENDING)
+------------------------------------------+----------+-------+-----------+---------------+-------------+--------------+
| Trial name                               | status   | loc   |   dropout |   hidden_size |          lr |   num_layers |
|------------------------------------------+----------+-------+-----------+---------------+-------------+--------------|
| train_rnn_with_attention_ray_23c00_00000 | PENDING  |       | 0.222107  |           512 | 0.00298953  |            2 |
| train_rnn_with_attention_ray_23c00_00001 | PENDING  |       | 0.465705  |        

(train_rnn_with_attention_ray pid=39511) 2025-06-04 20:04:45.236 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
 84%|████████▍ | 2189/2600 [00:01<00:00, 1977.96it/s]


== Status ==
Current time: 2025-06-04 20:04:46 (running for 00:00:10.29)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 4.0/4 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-06-04_20-04-33_195499_39160/artifacts/2025-06-04_20-04-36/train_rnn_with_attention_ray_2025-06-04_20-04-36/driver_artifacts
Number of trials: 10/10 (6 PENDING, 4 RUNNING)
+------------------------------------------+----------+-----------------+-----------+---------------+-------------+--------------+
| Trial name                               | status   | loc             |   dropout |   hidden_size |          lr |   num_layers |
|------------------------------------------+----------+-----------------+-----------+---------------+-------------+--------------|
| train_rnn_with_attention_ray_23c00_00000 | RUNNING  | 10.0.0.13:39510 | 0.222107  |           512 | 0.00298953  |            2 |
| train_rnn_with_attention_ray_

100%|██████████| 651/651 [00:00<00:00, 1878.69it/s]
(train_rnn_with_attention_ray pid=39511) 2025-06-04 20:04:47.221 | INFO     | mltrainer.settings:check_path:60 - Created logdir /tmp/ray/session_2025-06-04_20-04-33_195499_39160/artifacts/2025-06-04_20-04-36/train_rnn_with_attention_ray_2025-06-04_20-04-36/working_dirs/train_rnn_with_attention_ray_23c00_00001_1_dropout=0.4657,hidden_size=128,lr=0.0002,num_layers=4_2025-06-04_20-04-36/ray_logs
(train_rnn_with_attention_ray pid=39511) 2025-06-04 20:04:47.221 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to ray_logs/20250604-200447
100%|██████████| 651/651 [00:00<00:00, 1950.74it/s]
(train_rnn_with_attention_ray pid=39511) 2025-06-04 20:04:48.963 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
  0%|          | 0/81 [00:00<?, ?it/s]
(train_rnn_with_attention_ray pid=39511) 
  2%|▏         | 2/81 [00:00<00:04, 17.63it/s]
(train_rnn_with_attention_ra

...

== Status ==
Current time: 2025-06-04 20:07:38 (running for 00:03:01.96)
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: 0.928125
Logical resource usage: 1.0/4 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-06-04_20-04-33_195499_39160/artifacts/2025-06-04_20-04-36/train_rnn_with_attention_ray_2025-06-04_20-04-36/driver_artifacts
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+------------------------------------------+------------+-----------------+-----------+---------------+-------------+--------------+----------------------+
| Trial name                               | status     | loc             |   dropout |   hidden_size |          lr |   num_layers |   training_iteration |
|------------------------------------------+------------+-----------------+-----------+---------------+-------------+--------------+----------------------|
| train_rnn_with_attention_ray_23c00_00009 | RUNNING    | 10.0.0.13:40101 | 0.24

(train_rnn_with_attention_ray pid=40101) 
 40%|███▉      | 32/81 [00:01<00:01, 25.37it/s]
(train_rnn_with_attention_ray pid=40101) 
 43%|████▎     | 35/81 [00:01<00:01, 25.72it/s]
(train_rnn_with_attention_ray pid=40101) 
 47%|████▋     | 38/81 [00:01<00:01, 26.66it/s]
(train_rnn_with_attention_ray pid=40101) 
 51%|█████     | 41/81 [00:01<00:01, 26.74it/s]
(train_rnn_with_attention_ray pid=40101) 
 54%|█████▍    | 44/81 [00:01<00:01, 26.56it/s]
(train_rnn_with_attention_ray pid=40101) 
 58%|█████▊    | 47/81 [00:01<00:01, 26.77it/s]
(train_rnn_with_attention_ray pid=40101) 
 62%|██████▏   | 50/81 [00:01<00:01, 24.85it/s]
(train_rnn_with_attention_ray pid=40101) 
 65%|██████▌   | 53/81 [00:02<00:01, 25.42it/s]
(train_rnn_with_attention_ray pid=40101) 
 69%|██████▉   | 56/81 [00:02<00:01, 22.56it/s]
(train_rnn_with_attention_ray pid=40101) 
 73%|███████▎  | 59/81 [00:02<00:00, 23.35it/s]
(train_rnn_with_attention_ray pid=40101) 
 77%|███████▋  | 62/81 [00:02<00:00, 24.94it/s]
(train_rnn

== Status ==
Current time: 2025-06-04 20:07:43 (running for 00:03:06.99)
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: 0.928125
Logical resource usage: 1.0/4 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-06-04_20-04-33_195499_39160/artifacts/2025-06-04_20-04-36/train_rnn_with_attention_ray_2025-06-04_20-04-36/driver_artifacts
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+------------------------------------------+------------+-----------------+-----------+---------------+-------------+--------------+----------------------+
| Trial name                               | status     | loc             |   dropout |   hidden_size |          lr |   num_layers |   training_iteration |
|------------------------------------------+------------+-----------------+-----------+---------------+-------------+--------------+----------------------|
| train_rnn_with_attention_ray_23c00_00009 | RUNNING    | 10.0.0.13:40101 | 0.24

(train_rnn_with_attention_ray pid=40101) 
100%|██████████| 81/81 [00:03<00:00, 26.76it/s]
(train_rnn_with_attention_ray pid=40101) 2025-06-04 20:07:43.831 | INFO     | mltrainer.trainer:report:209 - Epoch 9 train 0.1506 test 0.2549 metric ['0.9219']
(train_rnn_with_attention_ray pid=40101) 2025-06-04 20:07:43.831 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.1094, current loss 0.2549.Counter 1/5.
100%|██████████| 10/10 [00:35<00:00,  3.55s/it]
2025-06-04 20:07:44,040	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/azureuser/ray_results/train_rnn_with_attention_ray_2025-06-04_20-04-36' in 0.0059s.
2025-06-04 20:07:44,045	INFO tune.py:1041 -- Total run time: 187.56 seconds (187.53 seconds for the tuning loop).


(train_rnn_with_attention_ray pid=40101) !!!!!!!!!!!!!! DEBUG metrics dict: {'Accuracy': 0.9203125}
== Status ==
Current time: 2025-06-04 20:07:44 (running for 00:03:07.53)
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: 0.92421875
Logical resource usage: 1.0/4 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-06-04_20-04-33_195499_39160/artifacts/2025-06-04_20-04-36/train_rnn_with_attention_ray_2025-06-04_20-04-36/driver_artifacts
Number of trials: 10/10 (10 TERMINATED)
+------------------------------------------+------------+-----------------+-----------+---------------+-------------+--------------+----------------------+
| Trial name                               | status     | loc             |   dropout |   hidden_size |          lr |   num_layers |   training_iteration |
|------------------------------------------+------------+-----------------+-----------+---------------+-------------+--------------+------------

In [11]:
best_result = result.get_best_config(metric="Accuracy", mode="max")

best_trial = result.get_best_trial(metric="Accuracy", mode="max", scope="all")
best_accuracy = best_trial.last_result["Accuracy"]

print(best_result)
print("Beste Accuracy:", best_accuracy)

{'hidden_size': 256, 'num_layers': 5, 'dropout': 0.1846677432423266, 'lr': 0.00028152890123466114}
Beste Accuracy: 0.96875
